In [1]:
import pandas as pd

In [5]:
def load_data():
    global X_train, y_train, X_test, y_test, X_train_all, X_test_all, X_all
    X_all = pd.read_csv("data/investigation_train_large_checked.csv")
    X_train_all = pd.read_csv("data/train.csv")
    X_test_all = pd.read_csv("data/test.csv")
    y_train = X_train_all['checked']
    y_test = X_test_all['checked']
    X_train = X_train_all.drop(columns=['checked'], errors='ignore', inplace=False)
    X_test = X_test_all.drop(columns=['checked'], errors='ignore', inplace=False)

In [6]:
load_data()
sensitive_features = [
    'persoon_geslacht_vrouw',
    'persoon_leeftijd_bij_onderzoek',
    'persoonlijke_eigenschappen_taaleis_voldaan'
]

In [9]:
correlation_rankings = {}

for sf in sensitive_features:
    # Compute correlations with the sensitive feature
    corr = (
        X_all.corr()[[sf]]
         .drop(sf)
         .sort_values(by=sf, key=lambda x: x.abs(), ascending=False)
    )

    correlation_rankings[sf] = corr

    print(f"\n=== Correlation ranking for {sf} ===")
    print(corr.head(10))  # show top 10


=== Correlation ranking for persoon_geslacht_vrouw ===
                                                   persoon_geslacht_vrouw
relatie_kind_leeftijd_verschil_ouder_eerste_kind                 0.368764
relatie_kind_huidige_aantal                                      0.261161
relatie_kind_heeft_kinderen                                      0.240015
relatie_kind_jongvolwassen                                       0.229253
relatie_overig_historie_vorm__onderhoudsplichtige                0.179374
relatie_kind_volwassen                                           0.147531
relatie_kind_tiener                                              0.143656
afspraak_signaal_voor_medewerker                                 0.126935
adres_aantal_brp_adres                                          -0.123538
contacten_onderwerp_terugbelverzoek                              0.122752

=== Correlation ranking for persoon_leeftijd_bij_onderzoek ===
                                                    persoon_leefti

In [10]:
correlation_rankings_asc = {}

for sf in sensitive_features:
    # Compute correlations with the sensitive feature
    corr = (
        X_all.corr()[[sf]]
         .drop(sf)
         .sort_values(by=sf, key=lambda x: x.abs(), ascending=True)
    )

    correlation_rankings_asc[sf] = corr

    print(f"\n=== Correlation ranking ascendingly for {sf} ===")
    print(corr.head(10))  # show top 10


=== Correlation ranking ascendingly for persoon_geslacht_vrouw ===
                                                    persoon_geslacht_vrouw
persoonlijke_eigenschappen_initiatief_opm                         0.000126
competentie_omgaan_met_verandering_en_aanpassen                  -0.000256
adres_recentste_wijk_kralingen_c                                 -0.000375
instrument_reden_beeindiging_historie_niet_succ...                0.000699
persoonlijke_eigenschappen_nl_schrijvenfalse                     -0.000766
instrument_reden_beeindiging_historie_overdrach...               -0.000860
competentie_instructies_en_procedures_opvolgen                   -0.001052
instrument_reden_beeindiging_historie_overdrach...               -0.001125
instrument_reden_beeindiging_historie_diagnose_...                0.001146
afspraak_toevoegen_inschrijving_uwvwb                            -0.001374

=== Correlation ranking ascendingly for persoon_leeftijd_bij_onderzoek ===
                               

In [15]:
import numpy as np

# Initialize table with all features
joint_corr = pd.DataFrame(index=X_all.columns)

for sf, corr_df in correlation_rankings_asc.items():
    # corr_df has one column: sf
    joint_corr[sf] = corr_df[sf]
joint_corr["joint_bias_score"] = np.prod(
    joint_corr[sensitive_features].abs(),
    axis=1
)
least_biased_features = (
    joint_corr
      .sort_values("joint_bias_score", ascending=False)
      .head(10)
      .index
      .tolist()
)

In [16]:
least_biased_features

['ontheffing_hist_ind',
 'pla_hist_pla_categorie_doelstelling_16',
 'persoonlijke_eigenschappen_taaleis_schrijfv_ok',
 'contacten_soort_telefoontje__uitgaand_',
 'contacten_onderwerp__arbeids_motivatie',
 'contacten_soort_document__uitgaand_',
 'contacten_soort_telefoontje__inkomend_',
 'contacten_onderwerp_terugbelverzoek',
 'pla_actueel_pla_categorie_doelstelling_16',
 'contacten_onderwerp_ziek__of_afmelding']